In [1]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] ="-1"
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
# import torchvision

#from keras.preprocessing.sequence import pad_sequences
#from keras.utils import to_categorical

from tqdm import tqdm_notebook as tqdm

import json
import nltk
from nltk import word_tokenize

from torch.utils.data.dataloader import default_collate


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
DIC_word_index = json.load(open("vocab.json", "r", encoding='utf-8'))
DIC_index_word = {index:word for word, index in DIC_word_index.items()}
word_vectors = np.load("wv_matrix100d.npy")

cuda


In [3]:
DIC_index_word[97]

'別'

In [4]:
def seq2sent(seq):
    return [DIC_index_word[i] for i in seq]

In [5]:
def to_words(ys):       
    tokens = []
    #print(ys)
    #input("")
    for catagorical_word in ys:
        index = np.argmax(catagorical_word.cpu().detach().numpy())
        #print(index)
        if index in DIC_index_word:
            tokens.append(DIC_index_word[index])
        else:
            tokens.append('<unk>');
    return tokens

In [6]:
def _run_iter(batch, training, model, loss_function):
    with torch.no_grad():
        e_x = batch['encoder_x'].long().to(device)
        d_x = batch['decoder_x'].long().to(device)
    output = model.forward(e_x, d_x)
    #print(output.shape)
    #print(batch['decoder_y'].shape)
    #input("")
    #print(output)
    #input("")
    loss = loss_function(output.view(-1, len(word_vectors)), batch['decoder_y'].view(-1).long().to(device))
    return output, loss

In [7]:
def _run_epoch(dataloader, training, model, optimizer, loss_function):
    model.train(training)
    if training:
        iter_in_epoch = min(len(dataloader), 1000000)
        description = 'train'
    else:
        iter_in_epoch = len(dataloader)
        description = 'test'
    grad_accumulate_steps = 1
    trange = tqdm(enumerate(dataloader), total=iter_in_epoch, desc=description)
    loss = 0
    for i, batch in trange:   
        if training and i >= iter_in_epoch:
            break

        if training:
            #print("batch:{}".format(batch))
            #print(batch['context'].dtype)
            optimizer.zero_grad()
            output, batch_loss = _run_iter(batch, training, model, loss_function)            
            
            batch_loss /= grad_accumulate_steps
            
            if i % grad_accumulate_steps == 0:
                optimizer.zero_grad()
            
            batch_loss.backward()

            if (i + 1) % grad_accumulate_steps == 0:
                optimizer.step()
            if((i+1) % 2000 == 0):
                print([DIC_index_word[i.item()] for i in batch['decoder_y'][0].cpu().detach()])
                print(to_words(output[0]))
                print(batch_loss)
        else:
            with torch.no_grad():
                output, batch_loss = _run_iter(batch, training, model, loss_function)
                if((i+1) % 2000 == 0):
                    print([DIC_index_word[i.item()] for i in batch['decoder_y'][0].cpu().detach()])
                    print(to_words(output[0]))
                
        loss += batch_loss.item()

    loss /= iter_in_epoch
    print('loss=%f\n' % loss)

In [8]:
def pad_post_zero(a, length):
    ret = []
    for _list in a:
        if(len(_list) < length):
            for ct in range(len(_list),length,1):
                _list.append(DIC_word_index["<pad>"])
        if(len(_list) > length):
            _list = _list[:length]
            
        ret.append(_list)
    return ret

In [ ]:
def to_categorical(y, num_classes=None, dtype='float32'):
    """Converts a class vector (integers) to binary class matrix.
    E.g. for use with categorical_crossentropy.
    # Arguments
        y: class vector to be converted into a matrix
            (integers from 0 to num_classes).
        num_classes: total number of classes.
        dtype: The data type expected by the input, as a string
            (`float32`, `float64`, `int32`...)
    # Returns
        A binary matrix representation of the input. The classes axis
        is placed last.
    # Example
    ```python
    # Consider an array of 5 labels out of a set of 3 classes {0, 1, 2}:
    > labels
    array([0, 2, 1, 2, 0])
    # `to_categorical` converts this into a matrix with as many
    # columns as there are classes. The number of rows
    # stays the same.
    > to_categorical(labels)
    array([[ 1.,  0.,  0.],
           [ 0.,  0.,  1.],
           [ 0.,  1.,  0.],
           [ 0.,  0.,  1.],
           [ 1.,  0.,  0.]], dtype=float32)
    ```
    """

    y = np.array(y, dtype='int')
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes), dtype=dtype)
    categorical[np.arange(n), y] = 1
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical


In [ ]:
def my_collate(datas):
    batch = {}
    # collate lists
    batch['decoder_x'] = torch.tensor([data['decoder_x'] for data in datas])
    batch['decoder_y'] = torch.tensor([data['decoder_y'] for data in datas])
    batch['encoder_x'] = torch.tensor([data['encoder_x'] for data in datas])
    return batch

In [ ]:
sent_len = 5

In [ ]:
encode_x = []
with open('sel_conversation/question.txt', 'r') as f:
    for l in f:
        l = l.split()
        tmp = []
        for word in l:
            try:
                index = DIC_word_index[word]
            except KeyError:
                index = DIC_word_index['<unk>']
            tmp.append(index)
        encode_x.append(tmp)        
encode_x = pad_post_zero(encode_x, sent_len)




In [ ]:
print(seq2sent(encode_x[73]))

['導致', '四個', '州', '停電', '<pad>']


In [ ]:
decode_x = []
decode_y = []
with open('sel_conversation/answer.txt', 'r') as f:
    for l in f:
        l = l.split()
        tmp = []
        for word in l:
            try:
                index = DIC_word_index[word]
            except KeyError:
                index = DIC_word_index['<unk>']
            tmp.append(index)
        decode_x.append([DIC_word_index['<bos>']] + tmp)    
        decode_y.append( tmp + [DIC_word_index['<eos>']])
    decode_x = pad_post_zero(decode_x, sent_len)
    decode_y = pad_post_zero(decode_y, sent_len)

In [ ]:
print(seq2sent(decode_x[68]))
print(seq2sent(decode_y[68]))

['<bos>', '在', '最新', '民調', '中']
['在', '最新', '民調', '中', '<eos>']


In [ ]:
datas = []
for a,b,c in zip(decode_x, decode_y , encode_x):
    data = {}

    data['decoder_x'] = a
    data['decoder_y'] = b;
    data['encoder_x'] = c;
    datas.append(data)

In [ ]:
#hyperparameters
lr = 1e-3  # learning rate
batch_size = 16
latent_dim = 256
iter_in_epoch = 500
embed_dim = word_vectors.shape[1]
print(word_vectors.shape)

(97841, 100)


In [ ]:
class SimpleRNN(torch.nn.Module):
    def __init__(self, latent_dim, seq_length, embed_dim, word_vectors):
        super(SimpleRNN, self).__init__()
        
        self.hidden_dim = latent_dim        
        self.seq_length = seq_length
        self.embed_dim = embed_dim
        
#         self.embedding = torch.nn.Embedding(len(word_vectors), embed_dim)
#         self.embedding.load_state_dict({'weight': word_vectors})
#         self.embedding.weight.requires_grad = False
#         self.embedding = torch.nn.Embedding.from_pretrained(torch.from_numpy(word_vectors),freeze=True)
        self.embedding = torch.nn.Embedding(len(word_vectors), embed_dim)
    
        self.GRU1 = torch.nn.GRU(self.embed_dim, latent_dim,  num_layers=1, batch_first = True)
        self.GRU2 = torch.nn.GRU(self.embed_dim, latent_dim,  num_layers=1, batch_first = True)
        
        self.Projection_layer = torch.nn.Sequential(
            torch.nn.Linear(latent_dim, len(word_vectors)),
            torch.nn.Softmax(-1),
        )
    def encoder_GRU(self, e_x):
        e_y, hiddens = self.GRU1(e_x)
        
        return e_y, hiddens
    
    def decoder_GRU(self, d_x, hiddens):
        d_y, hiddens2 = self.GRU2(d_x, hiddens)
        return d_y, hiddens2
    
    def forward(self, e_x, d_x):
        e_x = self.embedding(e_x).float()
        
        d_x = self.embedding(d_x).float()
        
        
        e_y, hiddens = self.encoder_GRU(e_x)
        
        first = True
        
#         for word in d_x.transpose(1,0):
#             word = word.view(word.shape[0], 1, -1)
#             d_y, hiddens = self.decoder_GRU(word, hiddens)
            
# #             print(d_y.shape)
#             if(first):
#                 first = False;
#                 last_layer_input = d_y
#             else:
#                 last_layer_input = torch.cat((last_layer_input, d_y), 1)
#         #print(last_layer_input.shape)
#         output = self.Projection_layer(last_layer_input)
        for word in d_x.transpose(1,0):
            word = word.view(word.shape[0], 1, -1)
            d_y, hiddens = self.decoder_GRU(word, hiddens)
            
            if(first):
                first = False;
                last_layer_input = self.Projection_layer(d_y)
            else:
                last_layer_input = torch.cat((last_layer_input, self.Projection_layer(d_y)), 1)

        output = last_layer_input
        #print(output)
        
        return output

In [ ]:
model = SimpleRNN(latent_dim, sent_len, embed_dim, word_vectors)

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=lr)
loss_function = torch.nn.CrossEntropyLoss()

In [ ]:
epoch = 0;
max_epochs = 100  # how many epochs to train for
while epoch < max_epochs:
    # train and evaluate train score
    print('training %i' % epoch)

    # train epoch
    dataloader = torch.utils.data.DataLoader(datas, batch_size = batch_size, shuffle = True, collate_fn = my_collate)
    log_train = _run_epoch(dataloader, True, model, optimizer, loss_function)

    # test epoch
    """
    print('evaluating %i' % epoch)
    dataloader = torch.utils.data.DataLoader(valid_datas, batch_size = batch_size, collate_fn=my_collate)
    log_valid = _run_epoch(dataloader, False, model, optimizer, loss_function)
    """
    epoch += 1

training 0


['什麼', '?', '<eos>', '<pad>', '<pad>']
['我', '<eos>', '<eos>', '<pad>', '<pad>']
tensor(11.2305, device='cuda:0', grad_fn=<DivBackward0>)
['好', '<eos>', '<pad>', '<pad>', '<pad>']
['我', '<eos>', '<pad>', '<pad>', '<pad>']
tensor(11.2037, device='cuda:0', grad_fn=<DivBackward0>)
['沒有', '太多', '的', '離開', '了']
['我', '<eos>', '<eos>', '<eos>', '<eos>']
tensor(11.2951, device='cuda:0', grad_fn=<DivBackward0>)
['好', '男人', ',', '成長', '起來']
['我', '<eos>', '<eos>', '<eos>', '我']
tensor(11.1875, device='cuda:0', grad_fn=<DivBackward0>)
['可是', '這樣', '要', '發現', '冰山']
['我', '<eos>', '<eos>', '<eos>', '<eos>']
tensor(11.2777, device='cuda:0', grad_fn=<DivBackward0>)
['我們', '是', '唯一', '的', '贏家']
['我', '<eos>', '我', '<eos>', '<eos>']
tensor(11.3551, device='cuda:0', grad_fn=<DivBackward0>)
['您', '不', '走', '我', '也']
['我', '的', '<eos>', '<eos>', '的']
tensor(11.3161, device='cuda:0', grad_fn=<DivBackward0>)
['對不起', '。', '<eos>', '<pad>', '<pad>']
['我', '<eos>', '<eos>', '<pad>', '<pad>']
tensor(11.2911, d

In [ ]:
print(DIC_word_index['<pad>'])